In [1]:
import argparse
import numpy as np
import os
import re
from random import shuffle
import eval.stats
import utils
from args import Args
import networkx as nx
args = Args()
pred_dir_input = "./"+"modified_graphs/" # args.dir_input + "/graphs/" 
test_dir_input = "./" + "graphs/"
model_name = 'GraphRNN_RNN' # self.model_name_all --> evaluation.py
pred_dataset_name = 'yelp' # self.dataset_name_all --> evaluation.py 
test_dataset_name = 'YELP' # self.dataset_name_all --> evaluation.py 
hidden = 128 # evaluate.py -> evaluation -> evaluation_epoch
epoch = 2 # we need the final epoch trained lstm performance on test graph to give the best predict 
sample_time = 1 # only one, because we use GRAPHRNN_RNN --> evaluate.p -> evaluation -> evaluation_epoch




fname_test = test_dir_input + model_name + '_' + test_dataset_name + '_' + str(args.num_layers) + '_' + str(hidden) + '_test_' + str(0) + '.dat'
fname_pred = pred_dir_input + model_name + '_' + pred_dataset_name + '_' + str(args.num_layers) + '_' + str(hidden) + '_pred_' + str(epoch) + '.dat'



graph_test = utils.load_graph_list(fname_test,is_real=True)
graph_pred = utils.load_graph_list(fname_pred,is_real=False)
graph_test_len = len(graph_test)
graph_train = graph_test[0:int(0.8 * graph_test_len)] # train
graph_validate = graph_test[0:int(0.2 * graph_test_len)] # validate
graph_test = graph_test[int(0.8 * graph_test_len):] # test on a hold out test set

## CLEAN GRAPH
def find_nearest_idx(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

# shuffle(graph_test)
# shuffle(graph_pred)

# get length
real_graph_len = np.array([len(graph_test[i]) for i in range(len(graph_test))])
pred_graph_len = np.array([len(graph_pred[i]) for i in range(len(graph_pred))])
# print(real_graph_len,len(pred_graph_len))

# select pred samples
# The number of nodes are sampled from the similar distribution as the training set
pred_graph_new = []
pred_graph_len_new = []
for value in real_graph_len:
    pred_idx = find_nearest_idx(pred_graph_len, value)
    pred_graph_new.append(graph_pred[pred_idx])
    pred_graph_len_new.append(pred_graph_len[pred_idx])

graph_pred = pred_graph_new
graph_real = graph_test
# min_len = min(len(graph_test),len(graph_pred))

num_of_matches = 0
pred_to_real_dic = {}
matches = 0
for idx in range(len(graph_pred)):
    real_nodes = graph_test[idx].nodes()
    pred_nodes = graph_pred[idx].nodes()
    print(len(real_nodes),' ',len(pred_nodes))
    if len(real_nodes) == len(pred_nodes):
        matches += 1
print(matches/len(graph_pred))    
    
# TP = 0
# FP = 0
# FN = 0

    
# for item in real_edges:
#     if item in pred_edges:
#         TP += 1
#     if item not in pred_edges:
#         FN += 1
# for item in pred_edges:
#     if item not in real_edges:
#         FP += 1
# precision = TP/(TP+FP)
# recall = TP/(TP+FN)
# F1_value = 2*precision*recall/(precision+recall)
# print('precision: ',precision,'recall: ',recall, 'F1-value: ',F1_value)


2   2
3   3
3   3
3   3
2   2
3   3
2   2
2   2
2   2
2   2
3   3
2   2
2   2
2   2
2   2
4   4
2   2
2   2
2   2
4   4
2   2
2   2
2   2
2   2
3   3
2   2
3   3
3   3
2   2
2   2
2   2
2   2
3   3
2   2
2   2
2   2
2   2
2   2
2   2
2   2
4   4
3   3
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
3   3
2   2
2   2
2   2
2   2
3   3
2   2
10   11
4   4
3   3
7   7
4   4
2   2
2   2
6   6
2   2
2   2
2   2
2   2
2   2
2   2
3   3
4   4
8   8
2   2
2   2
2   2
2   2
2   2
7   7
2   2
2   2
2   2
3   3
2   2
2   2
2   2
2   2
2   2
7   7
3   3
2   2
3   3
3   3
13   13
6   6
2   2
2   2
2   2
2   2
2   2
2   2
5   5
3   3
4   4
4   4
2   2
3   3
3   3
3   3
2   2
2   2
2   2
2   2
2   2
2   2
3   3
2   2
3   3
2   2
2   2
3   3
3   3
3   3
2   2
2   2
2   2
2   2
2   2
2   2
7   7
2   2
2   2
3   3
2   2
2   2
2   2
2   2
3   3
3   3
5   5
8   8
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
3   3
5   5
2   2
2   2
23   24
2   2
3   3
3   

In [ ]:
from train import *
from utils import *
from data import *

args = Args()
os.environ['CUDA_VISIBLE_DEVICES'] = str(args.cuda)
print('CUDA', args.cuda)
if args.clean_tensorboard:
    if os.path.isdir("tensorboard"):
        shutil.rmtree("tensorboard")
        
# time = strftime("%Y-%m-%d %H:%M:%S", gmtime())
# configure("tensorboard/run"+time, flush_secs=5)

graphs = create_graphs.create(args)
# split datasets
random.seed(123)
shuffle(graphs)
graphs_len = len(graphs)
graphs_test = graphs[int(0.8 * graphs_len):]
graphs_train = graphs[0:int(0.8*graphs_len)]
graphs_validate = graphs[0:int(0.2*graphs_len)]


args.max_num_node = max([graphs[i].number_of_nodes() for i in range(len(graphs))])
max_num_edge = max([graphs[i].number_of_edges() for i in range(len(graphs))])
min_num_edge = min([graphs[i].number_of_edges() for i in range(len(graphs))])



dataset = Graph_sequence_sampler_pytorch(graphs_train,max_prev_node=args.max_prev_node,max_num_node=args.max_num_node)
sample_strategy = torch.utils.data.sampler.WeightedRandomSampler([1.0 / len(dataset) for i in range(len(dataset))],
                                                                 num_samples=args.batch_size*args.batch_ratio, replacement=True)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, num_workers=args.num_workers,
                                           sampler=sample_strategy)
rnn = GRU_plain(input_size=args.max_prev_node, embedding_size=args.embedding_size_rnn,
                hidden_size=args.hidden_size_rnn, num_layers=args.num_layers, has_input=True,
                has_output=True, output_size=args.hidden_size_rnn_output).cuda()
output = GRU_plain(input_size=1, embedding_size=args.embedding_size_rnn_output,
                   hidden_size=args.hidden_size_rnn_output, num_layers=args.num_layers, has_input=True,
                   has_output=True, output_size=1).cuda()
epoch = 1

# initialize optimizer
optimizer_rnn = optim.Adam(list(rnn.parameters()), lr=args.lr)
optimizer_output = optim.Adam(list(output.parameters()), lr=args.lr)

scheduler_rnn = MultiStepLR(optimizer_rnn, milestones=args.milestones, gamma=args.lr_rate)
scheduler_output = MultiStepLR(optimizer_output, milestones=args.milestones, gamma=args.lr_rate)
while epoch<=args.epochs:
    rnn.train() #sets the model on train mode
    output.train()
    loss_sum = 0
    for batch_idx, data in enumerate(dataset_loader):
        rnn.zero_grad()
        output.zero_grad()
        x_unsorted = data['x'].float()
        y_unsorted = data['y'].float()
    print(y_unsorted.size(0), ' ',y_unsorted.size(1))
    print(x_unsorted.size(1),' ',x_unsorted.size(1))
    epoch += 1
    if epoch == 2:
        break
    

In [2]:
import numpy as np
x = np.ones(8)
print(x.dtype)

float64
